In [1]:
import os
os.getcwd()
os.chdir('C:\\python\\YES24\\deep_contextual_bandits\\bandits')

In [4]:
!python setup.py build

running build


In [7]:
!python setup.py install

zip_safe flag not set; analyzing archive contents...


running install
running bdist_egg
running egg_info
writing bandits.egg-info\PKG-INFO
writing dependency_links to bandits.egg-info\dependency_links.txt
writing requirements to bandits.egg-info\requires.txt
writing top-level names to bandits.egg-info\top_level.txt
reading manifest file 'bandits.egg-info\SOURCES.txt'
writing manifest file 'bandits.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
byte-compiling build\bdist.win-amd64\egg\algorithms\bb_alpha_divergence_model.py to bb_alpha_divergence_model.cpython-37.pyc
byte-compiling build\bdist.win-amd64\egg\algorithms\bf_variational_neural_bandit_model.py to bf_variational_neural_bandit_model.cpython-37.pyc
byte-compiling build\bdist.win-amd64\egg\algorithms\bootstrapped_bnn_sampling.py to bootstrapped_bnn_sampling.cpython-37.pyc
byte-compiling build\bdist.win-amd64\egg\algorithms\fixed_policy_sampling.py to fixed_policy_sampling.cpython-37.pyc
byte-compiling build\bdist.win-amd64\egg\algorit

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
from absl import app
from absl import flags
import numpy as np
import os
import tensorflow as tf
import sys
import pandas as pd

In [3]:
os.chdir('C:\\python\\YES24\\deep_contextual_bandits')

In [4]:
# Set up your file routes to the data files.
base_route = os.getcwd()
data_route = 'contextual_bandits/datasets'




flags.DEFINE_string('logdir', '/tmp/bandits/', 'Base directory to save output')
flags.DEFINE_string(
    'mushroom_data',
    os.path.join(base_route, data_route, 'mushroom.data'),
    'Directory where Mushroom data is stored.')
flags.DEFINE_string('f', '', 'kernel')
FLAGS = flags.FLAGS
FLAGS.set_default('alsologtostderr', True)
FLAGS(sys.argv)

['C:\\Users\\admin\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py']

In [29]:
flags.DEFINE_string(
    'jester_data',
    os.path.join(base_route, data_route, 'jester_data_40jokes_19181users.npy'),
    'Directory where Jester data is stored.')

In [5]:
flags.DEFINE_string(
    'census_data',
    os.path.join(base_route, data_route, 'USCensus1990.data.txt'),
    'Directory where Census data is stored.')

In [6]:
def sample_data(data_type, num_contexts=None):
    """Sample data from given 'data_type'.

  Args:
    data_type: Dataset from which to sample.
    num_contexts: Number of contexts to sample.

  Returns:
    dataset: Sampled matrix with rows: (context, reward_1, ..., reward_num_act).
    opt_rewards: Vector of expected optimal reward for each context.
    opt_actions: Vector of optimal action for each context.
    num_actions: Number of available actions.
    context_dim: Dimension of each context.
    """

    if data_type == 'linear':
        # Create linear dataset
        num_actions = 8
        context_dim = 10
        noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
        dataset, _, opt_linear = sample_linear_data(num_contexts, context_dim,
                                                num_actions, sigma=noise_stds)
        opt_rewards, opt_actions = opt_linear
    elif data_type == 'sparse_linear':
        # Create sparse linear dataset
        num_actions = 7
        context_dim = 10
        noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
        num_nnz_dims = int(context_dim / 3.0)
        dataset, _, opt_sparse_linear = sample_sparse_linear_data(
        num_contexts, context_dim, num_actions, num_nnz_dims, sigma=noise_stds)
        opt_rewards, opt_actions = opt_sparse_linear
    elif data_type == 'mushroom':
        # Create mushroom dataset
        num_actions = 2
        context_dim = 117
        file_name = FLAGS.mushroom_data
        dataset, opt_mushroom = sample_mushroom_data(file_name, num_contexts)
        opt_rewards, opt_actions = opt_mushroom
    elif data_type == 'financial':
        num_actions = 8
        context_dim = 21
        num_contexts = min(3713, num_contexts)
        noise_stds = [0.01 * (i + 1) for i in range(num_actions)]
        file_name = FLAGS.financial_data
        dataset, opt_financial = sample_stock_data(file_name, context_dim,
                                               num_actions, num_contexts,
                                               noise_stds, shuffle_rows=True)
        opt_rewards, opt_actions = opt_financial
    elif data_type == 'jester':
        num_actions = 8
        context_dim = 32
        num_contexts = min(19181, num_contexts)
        file_name = FLAGS.jester_data
        dataset, opt_jester = sample_jester_data(file_name, context_dim,
                                             num_actions, num_contexts,
                                             shuffle_rows=True,
                                             shuffle_cols=True)
        opt_rewards, opt_actions = opt_jester
    elif data_type == 'statlog':
        file_name = FLAGS.statlog_data
        num_actions = 7
        num_contexts = min(43500, num_contexts)
        sampled_vals = sample_statlog_data(file_name, num_contexts,
                                       shuffle_rows=True)
        contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
        dataset = np.hstack((contexts, rewards))
        context_dim = contexts.shape[1]
    elif data_type == 'adult':
        file_name = FLAGS.adult_data
        num_actions = 14
        num_contexts = min(45222, num_contexts)
        sampled_vals = sample_adult_data(file_name, num_contexts,
                                     shuffle_rows=True)
        contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
        dataset = np.hstack((contexts, rewards))
        context_dim = contexts.shape[1]
    elif data_type == 'covertype':
        file_name = FLAGS.covertype_data
        num_actions = 7
        num_contexts = min(150000, num_contexts)
        sampled_vals = sample_covertype_data(file_name, num_contexts,
                                         shuffle_rows=True)
        contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
        dataset = np.hstack((contexts, rewards))
        context_dim = contexts.shape[1]
    elif data_type == 'census':
        file_name = FLAGS.census_data
        num_actions = 9
        num_contexts = min(150000, num_contexts)
        sampled_vals = sample_census_data(file_name, num_contexts,
                                      shuffle_rows=True)
        contexts, rewards, (opt_rewards, opt_actions) = sampled_vals
        dataset = np.hstack((contexts, rewards))
        context_dim = contexts.shape[1]
    elif data_type == 'wheel':
        delta = 0.95
        num_actions = 5
        context_dim = 2
        mean_v = [1.0, 1.0, 1.0, 1.0, 1.2]
        std_v = [0.05, 0.05, 0.05, 0.05, 0.05]
        mu_large = 50
        std_large = 0.01
        dataset, opt_wheel = sample_wheel_bandit_data(num_contexts, delta,
                                                  mean_v, std_v,
                                                  mu_large, std_large)
        opt_rewards, opt_actions = opt_wheel

    return dataset, opt_rewards, opt_actions, num_actions, context_dim

In [27]:
def classification_to_bandit_problem(contexts, labels, num_actions=None):
    """Normalize contexts and encode deterministic rewards."""

    if num_actions is None:
        num_actions = np.max(labels) + 1
    num_contexts = contexts.shape[0]

    # Due to random subsampling in small problems, some features may be constant
    sstd = safe_std(np.std(contexts, axis=0, keepdims=True)[0, :])

    # Normalize features
    contexts = ((contexts - np.mean(contexts, axis=0, keepdims=True)) / sstd)

    # One hot encode labels as rewards
    rewards = np.zeros((num_contexts, num_actions))
    rewards[np.arange(num_contexts), labels] = 1.0

    return contexts, rewards, (np.ones(num_contexts), labels)


def safe_std(values):
    """Remove zero std values for ones."""
    return np.array([val if val != 0.0 else 1.0 for val in values])


def remove_underrepresented_classes(features, labels, thresh=0.0005):
    """Removes classes when number of datapoints fraction is below a threshold."""

    # Threshold doesn't seem to agree with https://arxiv.org/pdf/1706.04687.pdf
    # Example: for Covertype, they report 4 classes after filtering, we get 7?
    total_count = labels.shape[0]
    unique, counts = np.unique(labels, return_counts=True)
    ratios = counts.astype('float') / total_count
    vals_and_ratios = dict(zip(unique, ratios))
    print('Unique classes and their ratio of total: %s' % vals_and_ratios)
    keep = [vals_and_ratios[v] >= thresh for v in labels]
    return features[keep], labels[np.array(keep)]

In [7]:
def one_hot(df, cols):
    """Returns one-hot encoding of DataFrame df including columns in cols."""
    for col in cols:
        dummies = pd.get_dummies(df[col], prefix=col, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(col, axis=1)
    return df


def sample_mushroom_data(file_name,
                         num_contexts,
                         r_noeat=0,
                         r_eat_safe=5,
                         r_eat_poison_bad=-35,
                         r_eat_poison_good=5,
                         prob_poison_bad=0.5):
    """Samples bandit game from Mushroom UCI Dataset.

  Args:
    file_name: Route of file containing the original Mushroom UCI dataset.
    num_contexts: Number of points to sample, i.e. (context, action rewards).
    r_noeat: Reward for not eating a mushroom.
    r_eat_safe: Reward for eating a non-poisonous mushroom.
    r_eat_poison_bad: Reward for eating a poisonous mushroom if harmed.
    r_eat_poison_good: Reward for eating a poisonous mushroom if not harmed.
    prob_poison_bad: Probability of being harmed by eating a poisonous mushroom.

  Returns:
    dataset: Sampled matrix with n rows: (context, eat_reward, no_eat_reward).
    opt_vals: Vector of expected optimal (reward, action) for each context.

  We assume r_eat_safe > r_noeat, and r_eat_poison_good > r_eat_poison_bad.
      """
    # first two cols of df encode whether mushroom is edible or poisonous
    df = pd.read_csv(file_name, header=None)
    df = one_hot(df, df.columns)
    ind = np.random.choice(range(df.shape[0]), num_contexts, replace=True)

    contexts = df.iloc[ind, 2:]
    no_eat_reward = r_noeat * np.ones((num_contexts, 1))
    random_poison = np.random.choice(
      [r_eat_poison_bad, r_eat_poison_good],
      p=[prob_poison_bad, 1 - prob_poison_bad],
      size=num_contexts)
    eat_reward = r_eat_safe * df.iloc[ind, 0]
    eat_reward += np.multiply(random_poison, df.iloc[ind, 1])
    eat_reward = eat_reward.to_numpy().reshape((num_contexts, 1))

    # compute optimal expected reward and optimal actions
    exp_eat_poison_reward = r_eat_poison_bad * prob_poison_bad
    exp_eat_poison_reward += r_eat_poison_good * (1 - prob_poison_bad)
    opt_exp_reward = r_eat_safe * df.iloc[ind, 0] + max(
      r_noeat, exp_eat_poison_reward) * df.iloc[ind, 1]

    if r_noeat > exp_eat_poison_reward:
        # actions: no eat = 0 ; eat = 1
        opt_actions = df.iloc[ind, 0]  # indicator of edible
    else:
        # should always eat (higher expected reward)
        opt_actions = np.ones((num_contexts, 1))

    opt_vals = (opt_exp_reward.values, opt_actions.values)

    return np.hstack((contexts, no_eat_reward, eat_reward)), opt_vals




In [10]:
num_contexts = 2000
data_type = 'mushroom'
sampled_vals = sample_data(data_type, num_contexts)
dataset, opt_rewards, opt_actions, num_actions, context_dim = sampled_vals

In [11]:
sampled_vals

(array([[  0.,   0.,   1., ...,   0.,   0., -35.],
        [  0.,   0.,   0., ...,   0.,   0.,   5.],
        [  0.,   0.,   1., ...,   0.,   0.,   5.],
        ...,
        [  0.,   0.,   1., ...,   0.,   0.,   5.],
        [  0.,   0.,   0., ...,   0.,   0., -35.],
        [  0.,   0.,   0., ...,   0.,   0.,   5.]]),
 array([0, 0, 0, ..., 5, 0, 0], dtype=uint8),
 array([0, 0, 0, ..., 1, 0, 0], dtype=uint8),
 2,
 117)

In [27]:
set(opt_actions)

{0, 1}

In [8]:
num_contexts = 2000

In [12]:
file_name = FLAGS.mushroom_data

In [15]:
df = pd.read_csv(file_name,header=None)

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [17]:
df = one_hot(df, df.columns)

In [18]:
df

,0_e,0_p,1_b,1_c,1_f,1_k,1_s,1_x,2_f,2_g,...,21_s,21_v,21_y,22_d,22_g,22_l,22_m,22_p,22_u,22_w
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8120,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
8121,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [19]:
ind = np.random.choice(range(df.shape[0]), num_contexts, replace=True)

In [22]:
contexts = df.iloc[ind, 2:]

In [20]:
ind

array([6338,  687, 4088, ..., 6159, 4772, 3046])

In [28]:
def sample_jester_data(file_name, context_dim, num_actions, num_contexts,
                       shuffle_rows=True, shuffle_cols=False):
    """Samples bandit game from (user, joke) dense subset of Jester dataset.

  Args:
    file_name: Route of file containing the modified Jester dataset.
    context_dim: Context dimension (i.e. vector with some ratings from a user).
    num_actions: Number of actions (number of joke ratings to predict).
    num_contexts: Number of contexts to sample.
    shuffle_rows: If True, rows from original dataset are shuffled.
    shuffle_cols: Whether or not context/action jokes are randomly shuffled.

  Returns:
    dataset: Sampled matrix with rows: (context, rating_1, ..., rating_k).
    opt_vals: Vector of deterministic .ptimal (reward, action) for each context.
  """

    with tf.gfile.Open(file_name, 'rb') as f:
        dataset = np.load(f)

    if shuffle_cols:
        dataset = dataset[:, np.random.permutation(dataset.shape[1])]
    if shuffle_rows:
        np.random.shuffle(dataset)
    dataset = dataset[:num_contexts, :]

    assert context_dim + num_actions == dataset.shape[1], 'Wrong data dimensions.'

    opt_actions = np.argmax(dataset[:, context_dim:], axis=1)
    opt_rewards = np.array([dataset[i, context_dim + a]
                          for i, a in enumerate(opt_actions)])

    return dataset, (opt_rewards, opt_actions)

In [35]:
dataset, opt_jester = sample_jester_data(file_name, context_dim,
                                             num_actions, num_contexts,
                                             shuffle_rows=True,
                                             shuffle_cols=True)

In [56]:
opt_jester

(array([3.88, 4.22, 6.99, ..., 9.13, 2.96, 8.79]),
 array([3, 3, 3, ..., 3, 4, 5], dtype=int64))

In [58]:
dataset[:, context_dim:][0]

array([-8.4 ,  0.1 , -9.71,  3.88, -3.25,  0.92, -0.1 ,  2.33])

In [60]:
dataset[0, context_dim + 3]

3.88

In [57]:
np.argmax(dataset[:, context_dim:], axis=1)

array([3, 3, 3, ..., 3, 4, 5], dtype=int64)

In [63]:
opt_actions

array([3, 3, 3, ..., 3, 4, 5], dtype=int64)

In [62]:
opt_rewards

array([3.88, 4.22, 6.99, ..., 9.13, 2.96, 8.79])

In [36]:
opt_rewards, opt_actions = opt_jester

In [40]:
np.random.permutation(dataset.shape[1])

array([13, 28, 36,  2, 20, 34, 26, 30, 37, 14, 29,  4, 18, 24,  8, 27, 33,
        1, 32, 17, 15,  3,  0, 22, 39, 16, 35, 10, 21, 11, 31,  6, 23,  7,
        9, 38, 19,  5, 12, 25])

In [30]:
file_name = FLAGS.jester_data
with tf.gfile.Open(file_name, 'rb') as f:
    dataset = np.load(f)

In [33]:
num_actions = 8
context_dim = 32
num_contexts = min(19181, num_contexts)

In [34]:
num_contexts

2000

In [13]:
def sample_census_data(file_name, num_contexts, shuffle_rows=True,
                       remove_underrepresented=False):
    """Returns bandit problem dataset based on the UCI census data.

Args:
    file_name: Route of file containing the Census dataset.
    num_contexts: Number of contexts to sample.
    shuffle_rows: If True, rows from original dataset are shuffled.
    remove_underrepresented: If True, removes arms with very few rewards.

  Returns:
    dataset: Sampled matrix with rows: (context, action rewards).
    opt_vals: Vector of deterministic optimal (reward, action) for each context.

  Preprocessing:
    * drop rows with missing labels
    * convert categorical variables to 1 hot encoding

  Note: this is the processed (not the 'raw') dataset. It contains a subset
  of the raw features and they've all been discretized.

  https://archive.ics.uci.edu/ml/datasets/US+Census+Data+%281990%29
  """
# Note: this dataset is quite large. It will be slow to load and preprocess.
    with tf.gfile.Open(file_name, 'r') as f:
        df = (pd.read_csv(f, header=0, na_values=['?'])
          .dropna())

    num_actions = 9

    if shuffle_rows:
        df = df.sample(frac=1)
    df = df.iloc[:num_contexts, :]

    # Assuming what the paper calls response variable is the label?
    labels = df['dOccup'].astype('category').cat.codes.values
    # In addition to label, also drop the (unique?) key.
    df = df.drop(['dOccup', 'caseid'], axis=1)

    # All columns are categorical. Convert to 1 hot encoding.
    df = pd.get_dummies(df, columns=df.columns)

    if remove_underrepresented:
        df, labels = remove_underrepresented_classes(df, labels)
    contexts = df.values

    return classification_to_bandit_problem(contexts, labels, num_actions)


In [ ]:
def sample_stock_data(file_name, context_dim, num_actions, num_contexts,
                      sigma, shuffle_rows=True):
  """Samples linear bandit game from stock prices dataset.

  Args:
    file_name: Route of file containing the stock prices dataset.
    context_dim: Context dimension (i.e. vector with the price of each stock).
    num_actions: Number of actions (different linear portfolio strategies).
    num_contexts: Number of contexts to sample.
    sigma: Vector with additive noise levels for each action.
    shuffle_rows: If True, rows from original dataset are shuffled.

  Returns:
    dataset: Sampled matrix with rows: (context, reward_1, ..., reward_k).
    opt_vals: Vector of expected optimal (reward, action) for each context.
  """

  with tf.gfile.Open(file_name, 'r') as f:
    contexts = np.loadtxt(f, skiprows=1)

  if shuffle_rows:
    np.random.shuffle(contexts)
  contexts = contexts[:num_contexts, :]

  betas = np.random.uniform(-1, 1, (context_dim, num_actions))
  betas /= np.linalg.norm(betas, axis=0)

  mean_rewards = np.dot(contexts, betas)
  noise = np.random.normal(scale=sigma, size=mean_rewards.shape)
  rewards = mean_rewards + noise

  opt_actions = np.argmax(mean_rewards, axis=1)
  opt_rewards = [mean_rewards[i, a] for i, a in enumerate(opt_actions)]
  return np.hstack((contexts, rewards)), (np.array(opt_rewards), opt_actions)





def sample_statlog_data(file_name, num_contexts, shuffle_rows=True,
                        remove_underrepresented=False):
  """Returns bandit problem dataset based on the UCI statlog data.

  Args:
    file_name: Route of file containing the Statlog dataset.
    num_contexts: Number of contexts to sample.
    shuffle_rows: If True, rows from original dataset are shuffled.
    remove_underrepresented: If True, removes arms with very few rewards.

  Returns:
    dataset: Sampled matrix with rows: (context, action rewards).
    opt_vals: Vector of deterministic optimal (reward, action) for each context.

  https://archive.ics.uci.edu/ml/datasets/Statlog+(Shuttle)
  """

  with tf.gfile.Open(file_name, 'r') as f:
    data = np.loadtxt(f)

  num_actions = 7  # some of the actions are very rarely optimal.

  # Shuffle data
  if shuffle_rows:
    np.random.shuffle(data)
  data = data[:num_contexts, :]

  # Last column is label, rest are features
  contexts = data[:, :-1]
  labels = data[:, -1].astype(int) - 1  # convert to 0 based index

  if remove_underrepresented:
    contexts, labels = remove_underrepresented_classes(contexts, labels)

  return classification_to_bandit_problem(contexts, labels, num_actions)


def sample_adult_data(file_name, num_contexts, shuffle_rows=True,
                      remove_underrepresented=False):
  """Returns bandit problem dataset based on the UCI adult data.

  Args:
    file_name: Route of file containing the Adult dataset.
    num_contexts: Number of contexts to sample.
    shuffle_rows: If True, rows from original dataset are shuffled.
    remove_underrepresented: If True, removes arms with very few rewards.

  Returns:
    dataset: Sampled matrix with rows: (context, action rewards).
    opt_vals: Vector of deterministic optimal (reward, action) for each context.

  Preprocessing:
    * drop rows with missing values
    * convert categorical variables to 1 hot encoding

  https://archive.ics.uci.edu/ml/datasets/census+income
  """
  with tf.gfile.Open(file_name, 'r') as f:
    df = pd.read_csv(f, header=None,
                     na_values=[' ?']).dropna()

  num_actions = 14

  if shuffle_rows:
    df = df.sample(frac=1)
  df = df.iloc[:num_contexts, :]

  labels = df[6].astype('category').cat.codes.as_matrix()
  df = df.drop([6], axis=1)

  # Convert categorical variables to 1 hot encoding
  cols_to_transform = [1, 3, 5, 7, 8, 9, 13, 14]
  df = pd.get_dummies(df, columns=cols_to_transform)

  if remove_underrepresented:
    df, labels = remove_underrepresented_classes(df, labels)
  contexts = df.as_matrix()

  return classification_to_bandit_problem(contexts, labels, num_actions)




def sample_covertype_data(file_name, num_contexts, shuffle_rows=True,
                          remove_underrepresented=False):
  """Returns bandit problem dataset based on the UCI Cover_Type data.

  Args:
    file_name: Route of file containing the Covertype dataset.
    num_contexts: Number of contexts to sample.
    shuffle_rows: If True, rows from original dataset are shuffled.
    remove_underrepresented: If True, removes arms with very few rewards.

  Returns:
    dataset: Sampled matrix with rows: (context, action rewards).
    opt_vals: Vector of deterministic optimal (reward, action) for each context.

  Preprocessing:
    * drop rows with missing labels
    * convert categorical variables to 1 hot encoding

  https://archive.ics.uci.edu/ml/datasets/Covertype
  """
  with tf.gfile.Open(file_name, 'r') as f:
    df = (pd.read_csv(f, header=0, na_values=['?'])
          .dropna())

  num_actions = 7

  if shuffle_rows:
    df = df.sample(frac=1)
  df = df.iloc[:num_contexts, :]

  # Assuming what the paper calls response variable is the label?
  # Last column is label.
  labels = df[df.columns[-1]].astype('category').cat.codes.as_matrix()
  df = df.drop([df.columns[-1]], axis=1)

  # All columns are either quantitative or already converted to 1 hot.
  if remove_underrepresented:
    df, labels = remove_underrepresented_classes(df, labels)
  contexts = df.as_matrix()

  return classification_to_bandit_problem(contexts, labels, num_actions)




In [165]:
hparams = tf.contrib.training.HParams(num_actions=num_actions)

hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                               context_dim=context_dim2,
                                               a0=6,
                                               b0=6,
                                               lambda_prior=0.25,
                                               initial_pulls=5)

In [135]:
context_dim2

73

In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from scipy.stats import invgamma

from bandits.core.bandit_algorithm import BanditAlgorithm
from bandits.core.contextual_dataset import ContextualDataset

In [16]:
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

In [17]:
from bandits.core.contextual_bandit import run_contextual_bandit

In [166]:
algo = [LinearFullPosteriorSampling('LinFullPost', hparams_linear)]

In [167]:
t_init = time.time()
results = run_contextual_bandit(context_dim2, num_actions, dataset2, algo)

In [86]:
num_actions

8

In [75]:
results

(array([[0.],
        [1.],
        [2.],
        ...,
        [7.],
        [4.],
        [3.]]),
 array([[-8.4 ],
        [ 3.79],
        [ 3.54],
        ...,
        [ 5.49],
        [ 2.96],
        [ 5.19]]))

In [138]:
_, h_rewards = results

In [139]:
h_rewards

array([[0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],

In [51]:
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
    name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')

In [148]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, data_type)

NameError: name 'data_type' is not defined

In [11]:
file_name = FLAGS.census_data

In [153]:

num_actions = 5
num_contexts = 1000





In [19]:
with tf.gfile.Open(file_name, 'r') as f:
    df = (pd.read_csv(f, header=0, na_values=['?'])
          .dropna())

In [154]:
reward_raw = pd.read_csv("C:/python/YES24/reward_100000.csv")
reward_raw

,account_id,product_id,purchase,book_cluster,user_cluster
0,100001,88440267,1.0,3.0,3.0
1,100001,88440267,1.0,3.0,3.0
2,100001,85951536,1.0,3.0,3.0
3,100001,85951536,1.0,3.0,3.0
4,100001,86895523,0.0,3.0,3.0
...,...,...,...,...,...
99995,133498,89337980,1.0,3.0,3.0
99996,133498,90082148,0.0,3.0,3.0
99997,133498,68749139,0.0,4.0,3.0
99998,133498,68749139,0.0,4.0,3.0


In [171]:
reward_raw['purchase'].sum()

30651.0

In [173]:
import sys
sys.stdout.flush()

In [174]:
df2 = pd.read_csv('C:/python/YES24/context_5000000_0630.csv')
df2 = df2.iloc[:,1:]

MemoryError: Unable to allocate 2.68 GiB for an array with shape (72, 5000000) and data type float64

In [151]:
df2

,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,cat11.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [20]:
df

,caseid,dAge,dAncstry1,dAncstry2,iAvail,iCitizen,iClass,dDepart,iDisabl1,iDisabl2,...,iTmpabsnt,dTravtime,iVietnam,dWeek89,iWork89,iWorklwk,iWWII,iYearsch,iYearwrk,dYrsserv
0,10000,5,0,1,0,0,5,3,2,2,...,0,5,0,2,1,1,0,11,1,0
1,10001,6,1,1,0,0,7,5,2,2,...,0,1,0,2,1,1,0,5,1,0
2,10002,3,1,2,0,0,7,4,2,2,...,0,2,0,2,1,1,0,10,1,0
3,10003,4,1,2,0,0,1,3,2,2,...,0,1,0,1,1,1,0,10,1,0
4,10004,7,1,1,0,0,0,0,2,2,...,3,0,0,0,2,2,0,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458280,2468280,7,1,2,0,0,0,0,2,2,...,3,0,0,0,2,2,1,7,5,1
2458281,2468281,1,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,0,0
2458282,2468282,3,3,1,0,0,1,2,2,2,...,0,5,0,2,1,1,0,11,1,0
2458283,2468283,6,0,1,0,0,1,3,2,2,...,0,1,0,2,1,1,0,10,1,0


In [152]:
num_actions= 9

In [22]:
df = df.sample(frac=1)
df = df.iloc[:num_contexts, :]

In [116]:

df2 = df2.iloc[:num_contexts, :]

In [117]:
df2

,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,cat11.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [155]:
labels2= reward_raw['book_cluster'].astype('int8').values

In [23]:
# Assuming what the paper calls response variable is the label?
labels = df['dOccup'].astype('category').cat.codes.values
# In addition to label, also drop the (unique?) key.
df = df.drop(['dOccup', 'caseid'], axis=1)

In [24]:
df = pd.get_dummies(df, columns=df.columns)

In [123]:
len(labels2)

100000

In [156]:
df2, labels2 = remove_underrepresented_classes(df2, labels2,0.001)

Unique classes and their ratio of total: {0: 0.12004, 1: 0.21227, 2: 0.14777, 3: 0.34609, 4: 0.17383}


In [157]:
df2

,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,cat11.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [96]:
labels2

array([3, 3, 3, ..., 4, 4, 3], dtype=int8)

In [158]:
contexts2 = df2.values

In [25]:
df

,dAge_0,dAge_1,dAge_2,dAge_3,dAge_4,dAge_5,dAge_6,dAge_7,dAncstry1_0,dAncstry1_1,...,iYearwrk_1,iYearwrk_2,iYearwrk_3,iYearwrk_4,iYearwrk_5,iYearwrk_6,iYearwrk_7,dYrsserv_0,dYrsserv_1,dYrsserv_2
2021524,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1080166,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
538098,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
782271,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2296178,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291602,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1912876,0,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
2352506,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2138635,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [28]:
df,labels = remove_underrepresented_classes(df, labels,0.005)

Unique classes and their ratio of total: {0: 0.41, 1: 0.15, 2: 0.179, 3: 0.084, 4: 0.013, 5: 0.066, 6: 0.092, 7: 0.0025, 8: 0.0035}


In [69]:
contexts

array([[ 7.46993976, -0.4662524 , -0.32696213, ...,  0.34641016,
        -0.31552737, -0.12992227],
       [-0.13386989,  2.14476106, -0.32696213, ...,  0.34641016,
        -0.31552737, -0.12992227],
       [-0.13386989, -0.4662524 , -0.32696213, ..., -2.88675135,
         3.16929715, -0.12992227],
       ...,
       [-0.13386989,  2.14476106, -0.32696213, ...,  0.34641016,
        -0.31552737, -0.12992227],
       [-0.13386989, -0.4662524 , -0.32696213, ...,  0.34641016,
        -0.31552737, -0.12992227],
       [-0.13386989, -0.4662524 , -0.32696213, ...,  0.34641016,
        -0.31552737, -0.12992227]])

In [159]:
num_contexts = contexts2.shape[0]
contexts2

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [160]:
num_contexts

100000

In [161]:
sampled_vals2 = classification_to_bandit_problem(contexts2, labels2, num_actions)

In [162]:
contexts2, rewards2, (opt_rewards2, opt_actions2) = sampled_vals2

In [163]:
dataset2 = np.hstack((contexts2, rewards2))

In [164]:
context_dim2 = contexts2.shape[1]

In [29]:
contexts = df.values

In [30]:
num_contexts = contexts.shape[0]

In [31]:
sstd = safe_std(np.std(contexts, axis=0, keepdims=True)[0, :])


In [32]:
contexts = ((contexts - np.mean(contexts, axis=0, keepdims=True)) / sstd)

In [33]:
contexts

array([[-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
        -0.33165406, -0.13580362],
       [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
        -0.33165406, -0.13580362],
       [-0.12585934, -0.46465271,  3.20865307, ...,  0.3643673 ,
        -0.33165406, -0.13580362],
       ...,
       [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
        -0.33165406, -0.13580362],
       [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
        -0.33165406, -0.13580362],
       [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
        -0.33165406, -0.13580362]])

In [34]:
rewards = np.zeros((num_contexts, num_actions))

In [35]:
rewards[np.arange(num_contexts), labels] = 1.0

In [36]:
rewards

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [37]:
labels

array([1, 0, 0, ..., 4, 1, 2], dtype=int8)

In [38]:
classification_to_bandit_problem(contexts, labels, num_actions)

(array([[-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
         -0.33165406, -0.13580362],
        [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
         -0.33165406, -0.13580362],
        [-0.12585934, -0.46465271,  3.20865307, ...,  0.3643673 ,
         -0.33165406, -0.13580362],
        ...,
        [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
         -0.33165406, -0.13580362],
        [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
         -0.33165406, -0.13580362],
        [-0.12585934, -0.46465271, -0.31165725, ...,  0.3643673 ,
         -0.33165406, -0.13580362]]),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]),
 (array([1., 1., 1., ..., 1., 1., 1.]),
  array([1, 0, 0, ..., 4, 1, 2], dtype=int8)))

In [54]:
sampled_vals = sample_census_data(file_name, num_contexts,
                                      shuffle_rows=True)

KeyboardInterrupt: 

In [59]:
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals

In [60]:
opt_actions

array([0, 0, 1, ..., 0, 1, 6], dtype=int8)

In [44]:

dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [207]:
class LinearFullPosteriorSampling(BanditAlgorithm):
 
    def __init__(self, name, hparams):
   

        self.name = name
        self.hparams = hparams

        # Gaussian prior for each beta_i
        self._lambda_prior = self.hparams.lambda_prior

        self.mu = [
            np.zeros(self.hparams.context_dim + 1)
            for _ in range(self.hparams.num_actions)
        ]

        self.cov = [(1.0 / self.lambda_prior) * np.eye(self.hparams.context_dim + 1)
                for _ in range(self.hparams.num_actions)]

        self.precision = [
            self.lambda_prior * np.eye(self.hparams.context_dim + 1)
            for _ in range(self.hparams.num_actions)
        ]

        # Inverse Gamma prior for each sigma2_i
        self._a0 = self.hparams.a0
        self._b0 = self.hparams.b0

        self.a = [self._a0 for _ in range(self.hparams.num_actions)]
        self.b = [self._b0 for _ in range(self.hparams.num_actions)]

        self.t = 0
        self.data_h = ContextualDataset(hparams.context_dim,
                                    hparams.num_actions,
                                    intercept=True)

    def action(self, context):
   
    # Round robin until each action has been selected "initial_pulls" times
        if self.t < self.hparams.num_actions * self.hparams.initial_pulls:
            return self.t % self.hparams.num_actions

        # Sample sigma2, and beta conditional on sigma2
        sigma2_s = [
            self.b[i] * invgamma.rvs(self.a[i])
            for i in range(self.hparams.num_actions)
        ]

        try:
            beta_s = [
              np.random.multivariate_normal(self.mu[i], sigma2_s[i] * self.cov[i])
              for i in range(self.hparams.num_actions)
            ]
        except np.linalg.LinAlgError as e:
          # Sampling could fail if covariance is not positive definite
            print('Exception when sampling from {}.'.format(self.name))
            print('Details: {} | {}.'.format(e.with_traceback, e.args))
            d = self.hparams.context_dim + 1
            beta_s = [
                np.random.multivariate_normal(np.zeros((d)), np.eye(d))
                for i in range(self.hparams.num_actions)
            ]

        # Compute sampled expected values, intercept is last component of beta
        vals = [
            np.dot(beta_s[i][:-1], context.T) + beta_s[i][-1]
            for i in range(self.hparams.num_actions)
        ]

        return np.argmax(vals)

    def update(self, context, action, reward):
  

        self.t += 1
        self.data_h.add(context, action, reward)

        # Update posterior of action with formulas: \beta | x,y ~ N(mu_q, cov_q)
        x, y = self.data_h.get_data(action)

        # The algorithm could be improved with sequential update formulas (cheaper)
        s = np.dot(x.T, x)

        # Some terms are removed as we assume prior mu_0 = 0.
        precision_a = s + self.lambda_prior * np.eye(self.hparams.context_dim + 1)
        cov_a = np.linalg.inv(precision_a)
        mu_a = np.dot(cov_a, np.dot(x.T, y))

        # Inverse Gamma posterior update
        a_post = self.a0 + x.shape[0] / 2.0
        b_upd = 0.5 * (np.dot(y.T, y) - np.dot(mu_a.T, np.dot(precision_a, mu_a)))
        b_post = self.b0 + b_upd

        # Store new posterior distributions
        self.mu[action] = mu_a
        self.cov[action] = cov_a
        self.precision[action] = precision_a
        self.a[action] = a_post
        self.b[action] = b_post

    @property
    def a0(self):
        return self._a0

    @property
    def b0(self):
        return self._b0

    @property
    def lambda_prior(self):
        return self._lambda_prior

In [213]:
def run_contextual_bandit(context_dim, num_actions, dataset, algos):
   
    num_contexts = dataset.shape[0]

    # Create contextual bandit
    cmab = ContextualBandit(context_dim, num_actions)
    cmab.feed_data(dataset)

    h_actions = np.empty((0, len(algos)), float)
    h_rewards = np.empty((0, len(algos)), float)

    # Run the contextual bandit process
    for i in range(num_contexts):
        context = cmab.context(i)
        actions = [a.action(context) for a in algos]
        rewards = [cmab.reward(i, action) for action in actions]

        for j, a in enumerate(algos):
            a.update(context, actions[j], rewards[j])

        h_actions = np.vstack((h_actions, np.array(actions)))
        h_rewards = np.vstack((h_rewards, np.array(rewards)))

    return h_actions, h_rewards


class ContextualBandit(object):
    

    def __init__(self, context_dim, num_actions):
   

        self._context_dim = context_dim
        self._num_actions = num_actions

    def feed_data(self, data):
      

        if data.shape[1] != self.context_dim + self.num_actions:
            raise ValueError('Data dimensions do not match.')

        self._number_contexts = data.shape[0]
        self.data = data
        self.order = range(self.number_contexts)

    def reset(self):
        """Randomly shuffle the order of the contexts to deliver."""
        self.order = np.random.permutation(self.number_contexts)

    def context(self, number):
        """Returns the number-th context."""
        return self.data[self.order[number]][:self.context_dim]

    def reward(self, number, action):
        """Returns the reward for the number-th context and action."""
        return self.data[self.order[number]][self.context_dim + action]

    def optimal(self, number):
        """Returns the optimal action (in hindsight) for the number-th context."""
        return np.argmax(self.data[self.order[number]][self.context_dim:])

    @property
    def context_dim(self):
        return self._context_dim

    @property
    def num_actions(self):
        return self._num_actions

    @property
    def number_contexts(self):
        return self._number_contexts

In [48]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algo)

C:\python\YES24\deep_contextual_bandits\bandits\algorithms\linear_full_posterior_sampling.py:99: RuntimeWarning: covariance is not positive-semidefinite.
  for i in range(self.hparams.num_actions)


Exception when sampling from LinFullPost.
Details: <built-in method with_traceback of LinAlgError object at 0x000002557A8E9DC8> | ('SVD did not converge',).
Exception when sampling from LinFullPost.
Details: <built-in method with_traceback of LinAlgError object at 0x000002557A8E9DC8> | ('SVD did not converge',).
Exception when sampling from LinFullPost.
Details: <built-in method with_traceback of LinAlgError object at 0x000002557A8E9DC8> | ('SVD did not converge',).


In [195]:
num_actions

9

In [200]:
np.linalg.LinAlgError.with_traceback.

<method 'with_traceback' of 'BaseException' objects>

In [141]:
_

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [3.],
       [2.],
       [4.],
       [3.],
       [4.],
       [4.],
       [4.],
       [1.],
       [1.],
       [4.],
       [2.],
       [3.],
       [0.],
       [1.],
       [1.],
       [4.],
       [4.],
       [1.],
       [2.],
       [0.],
       [2.],
       [4.],
       [3.],
       [3.],
       [1.],
       [0.],
       [3.],
       [3.],
       [4.],
       [2.],
       [2.],
       [4.],
       [0.],
       [3.],
       [4.],
       [0.],
       [4.],
       [0.],
       [4.],
       [0.],
       [3.],
       [1.],
       [2.],
       [1.],
       [4.],
       [1.],
       [4.],
       [2.],
       [3.],
       [3.],
       [4.],
       [1.],
       [2.],
       [1.],
       [2.],
       [3.],
       [1.],
       [0.],
       [4.],
       [1.],
       [3.],
       [0.],
       [0.],
       [0.],
       [2.],
       [4.],
       [3.],

In [168]:
_, h_rewards = results

In [169]:
display_results(algo, opt_rewards2, opt_actions2, h_rewards, t_init, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 5877.081946372986 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =    97463.0.
---------------------------------------------------
Optimal total reward = 100000.0.
Frequency of optimal actions (action, frequency):
[[0, 12004], [1, 21227], [2, 14777], [3, 34609], [4, 17383]]
---------------------------------------------------
---------------------------------------------------


In [72]:
df = pd.read_csv('C:/python/YES24/full_context.csv',nrows=1000)

In [73]:
df

,index,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,0,0.000000,0.340515,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.171419,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.36621,0.000000
1,1,0.000000,0.340515,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.171419,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.36621,0.000000
2,2,0.000000,0.340515,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.000000,0.119370,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.179412
3,3,0.000000,0.340515,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.000000,0.119370,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.179412
4,4,0.000000,0.340515,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.171419,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.36621,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.661902,0.000000,0.0,0.0,0.0,0.0,0.333293,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.236211,0.000000,0.000000,0.263816,0.00000,0.000000
996,996,0.659485,0.000000,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.000000,0.129505,0.0,0.000000,0.000000,0.160154,0.000000,0.00000,0.000000
997,997,0.659485,0.000000,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,...,0.0,0.191098,0.000000,0.0,0.000000,0.000000,0.160154,0.000000,0.00000,0.000000
998,998,0.661902,0.000000,0.0,0.0,0.0,0.0,0.333293,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.236211,0.000000,0.000000,0.263816,0.00000,0.000000
